<a href="https://colab.research.google.com/github/valesierrai/Analisis-Supervisado-para-el-AD/blob/main/Perceptro%CC%81n_y_Adaline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Perceptrón


El perceptrón es uno de los modelos más simples de redes neuronales. Es un clasificador binario que toma una decisión lineal para separar dos clases. Utiliza una función de activación escalón para determinar la salida. Debería ofrecer una precisión alta si los datos son linealmente separables.

En este caso, podemos usar datos simples como la clasificación de transacciones contables en "gastos" (0) e e"ingresos" (1).

| Montos | Tipo |
|--------|------|
| 500    | 1    | (Ingreso)
| 300    | 1    | (Ingreso)
| -200   | 0    | (Gasto)
| -150   | 0    | (Gasto)

## Implementación en `tensorflow`

In [ ]:
import tensorflow as tf
import numpy as np

# Datos del ejemplo, creamos los datos de entrada (montos) y la etiqueta (tipos)
montos = np.array([[500], [300], [-200], [-150]], dtype=np.float32)
tipos = np.array([[1], [1], [0], [0]], dtype=np.float32)

# Creamos el modelo de perceptrón. units el número de neuronas. El perceptrón tiene 1
# La función de activación debería ser escalón, pero keras no la tiene...usaremos la más similar
modelo = tf.keras.Sequential([tf.keras.layers.Dense(units=1, activation = 'sigmoid')])

# Compilamos el modelo. El optimizador será el que se encargue de actualizar
# los pesos, en este caso será gradiente descendente estocástico
modelo.compile(optimizer = 'sgd', loss = 'binary_crossentropy',
               metrics = ['accuracy'])

# Entrenamos el modelo modelo.
# Una época significa entrenar la red neuronal con todos los datos
# de entrenamiento para un ciclo . En una época, usamos todos los datos exactamente una vez.
# Un pase hacia adelante para el cálculo de la salida y un pase hacia atrás para el cálculo del error,
# juntos se cuentan como un pase.
# Una época se compone de uno o más lotes, donde usamos una parte del conjunto de datos para
# entrenar la red neuronal.
modelo.fit(montos, tipos, epochs=10)

# Calculamos la precisión (accuracy)
predicted = modelo.predict(montos)
predicted_classes = (predicted >= 0.5).astype(int)
accuracy = np.mean(predicted_classes.flatten() == tipos.flatten())
print(accuracy)

# Evaluamos el modelo con el error de entrenamiento
predicted = modelo.predict(montos)
predicted_classes = (predicted >= 0.5).astype(int)  # Umbral para clasificación (0.5))
print(predicted_classes)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 503ms/step - accuracy: 0.0000e+00 - loss: 217.8757
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 1.0000 - loss: 0.0000e+00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[1]
 [1]
 [0]
 [0]]


## Implementación en `Pytorch`

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# hacemos tensores con los datos de entrada y la variable objetivo
X = torch.tensor(montos, dtype=torch.float32)

y = torch.tensor(tipos, dtype=torch.float32)

# Hacemos la clase perceptrón. En el __init__ están los atributos de la clase
# las demás funciones son llamados métodos
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()
        self.linear = nn.Linear(1, 1) #define las entradas y las neuronas en cada capa.
                                      #tenemos solo una caracterítica a la entrada

    def forward(self, x): #es el método para aplicar un sigmoide a la salida. La idea sería un escalón
        return torch.sigmoid(self.linear(x))

# Definimos un objeto de la clase Perceptrón
model_pytorch = Perceptron()

# Establecemos el criterio de pérdida (Binary Cross Entropy Loss) y
# el optimizador (gradiente descendente)
criterion = nn.BCELoss()
optimizer = optim.SGD(model_pytorch.parameters(), lr=0.01) #lr es la tasa de aprendizaje

# Hacemos el entrenamiento con 100 épocas
for epoch in range(100):
    optimizer.zero_grad() # limpiar los gradientes anteriores
    outputs = model_pytorch(X) #entrenamos
    loss = criterion(outputs, y) #calculamos la pérdida
    loss.backward() #hacemos el backward para calcular los nuevos gradientes
    optimizer.step() #ajustamos los pesos del modelo usando los gradientes calculados en loss.backward()

# Evaluamos la red con el mismo conjunto de entrenamiento (acierto de entrenamiento = 1- error de entrenamiento)
with torch.no_grad():
    predicted = model_pytorch(X) #realizamos la predicción
    predicted_classes = (predicted >= 0.5).float() #establecemos el umbral
    accuracy = (predicted_classes.eq(y).sum().item()) / y.size(0) #calculamos el acierto
    print(f'Perceptrón (PyTorch) - Precisión: {accuracy*100:.2f}%')

Perceptrón (PyTorch) - Precisión: 100.00%


# Adaline  (ADAptive LInear NEuron)

Adaline es similar al perceptrón, pero en lugar de usar una función de activación escalón, utiliza una función de activación lineal y minimiza el error cuadrático medio. Esto lo hace más adecuado para problemas de regresión y clasificación con gradiente descendente.

Vamos a crear un conjunto de datos relacionado con la aprobación de préstamos. Las características serán:

- Ingresos mensuales (en miles de dólares)
- Deuda actual (en miles de dólares)
- Historial de crédito (1: bueno, 0: malo)

|Ingresos|	Deuda	|Historial	|Aprobado|
|--------|--------|-----------|--------|
|5	|1.5	|1|	1|
|3	|2.0	|0	|0|
|4	|1.0	|1	|1|
|2	|2.5	|0	|0|
|6	|1.2	|1	|1|
|3.5|	2.2	|0	|0|

In [ ]:
# Cargamos las librería
import tensorflow as tf
import numpy as np

# Definimos la matriz de datos o características
X = np.array([
    [5, 1.5, 1],
    [3, 2.0, 0],
    [4, 1.0, 1],
    [2, 2.5, 0],
    [6, 1.2, 1],
    [3.5, 2.2, 0]
], dtype=float)

# Nuestra característica objetivo será la columna Aprobado
y = np.array([1, 0, 1, 0, 1, 0], dtype=float)


## Implementación con `tensorflow`

In [ ]:
# Instanciamos el modelo Adaline. Aunque no existe directamente, se puede generar con la función de activación lineal.
# Tenemos tres entradas (Ingreso, Deuda, Historial)
model_adaline = tf.keras.Sequential([tf.keras.layers.Dense(1, activation='linear', input_shape=(3,))])

# Compilamos nuestro modelo. Para Adaline la función de pérdida está dada por el MSE
model_adaline.compile(optimizer='sgd',
                      loss='mean_squared_error',
                      metrics=['mean_squared_error'])

# Entrenamos el modelo. Al establecer el parámetro 'verbose' en  0, 1 o 2, simplemente dices cómo deseas "ver" el progreso del entrenamiento para cada época.
model_adaline.fit(X, y, epochs=200, verbose=1)

# Evaluamos el modelo con el error de entrenamiento
mse = model_adaline.evaluate(X, y, verbose=0)
print(f'Adaline - MSE: {mse[1]:.4f}')

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 415ms/step - loss: 0.4638 - mean_squared_error: 0.4638
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.4433 - mean_squared_error: 0.4433
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.4266 - mean_squared_error: 0.4266
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.4115 - mean_squared_error: 0.4115
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.3975 - mean_squared_error: 0.3975
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.3841 - mean_squared_error: 0.3841
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.3713 - mean_squared_error: 0.3713
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.3591 - mean_squared_error: 0.3591
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.3474 - mean_squared_error: 0.3474
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.3362 - mean_squared_error: 0.3362
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss:

## Implementación con `Pytorch`

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Declaramos los datos como tensores, tanto en la matriz de características
X = torch.tensor([
    [5, 1.5, 1],
    [3, 2.0, 0],
    [4, 1.0, 1],
    [2, 2.5, 0],
    [6, 1.2, 1],
    [3.5, 2.2, 0]
], dtype=torch.float32)


# como la de la característica objetivo
y = torch.tensor([[1], [0], [1], [0], [1], [0]], dtype=torch.float32)

In [ ]:
# Hacemos algo muy similar al caso del Perceptrón. Creamos la clase Adaline que
# hereda de nn.Module.

class Adaline(nn.Module):
    def __init__(self):
        super(Adaline, self).__init__()
        self.linear = nn.Linear(3, 1) #tenemos 3 entradas y solo una neurona operando

    def forward(self, x):
        return self.linear(x) #tenemos la función de activación lineal a la salida

model_adaline_pytorch = Adaline() #creamos el objeto model_adaline_pytorch de la clase Adaline

# Definimos la función de pérdida con el MSE y el optimizador de gradiente descendente estocástico
criterion_adaline = nn.MSELoss()
optimizer_adaline = optim.SGD(model_adaline_pytorch.parameters(), lr=0.01) #como hiperpámetro está la tasa de aprendizaje

# Entrenamos el modelo como en el caso del perceptrón
for epoch in range(100):
    optimizer_adaline.zero_grad()
    outputs = model_adaline_pytorch(X)
    loss = criterion_adaline(outputs, y)
    loss.backward()
    optimizer_adaline.step()

# Evaluamos el modelo con el error de entrenamiento y el MSE
with torch.no_grad():
    mse = criterion_adaline(model_adaline_pytorch(X), y)
    print(f'Adaline (PyTorch) - MSE: {mse.item():.4f}')

Adaline (PyTorch) - MSE: 0.0152
